In [50]:
import pandas  as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

In [51]:
train_policies_clean = pd.read_csv('../data/train_policies_clean.csv', index_col=0)
test_policies_clean = pd.read_csv('../data/test_policies_clean.csv', index_col=0)

# prep raw test data
raw_test = pd.read_csv('../data/raw_test.csv', index_col=0)
raw_test = raw_test.set_index('policy_id')

In [52]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
y_train = train_policies_clean['convert_ind']
X_train = train_policies_clean.loc[:, train_policies_clean.columns != 'convert_ind']
# X_test = raw_test.loc[:, :]
X_test = test_policies_clean.loc[:, :]

In [53]:
import statsmodels.api as sm
logit_model=sm.Logit(y_train, X_train)
result=logit_model.fit()
# print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.338973
         Iterations 9


In [54]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train == 1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train == 0)))
  
# import SMOTE module from imblearn library
# pip install imblearn (if you don't have imblearn in your system)
from imblearn.over_sampling import SMOTE
oversample = SMOTE(random_state = 2)
X_train_res, y_train_res = oversample.fit_resample(X_train, y_train.ravel())
  
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))
  
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0)))

Before OverSampling, counts of label '1': 3570
Before OverSampling, counts of label '0': 28182 

After OverSampling, the shape of train_X: (56364, 28)
After OverSampling, the shape of train_y: (56364,) 

After OverSampling, counts of label '1': 28182
After OverSampling, counts of label '0': 28182


In [57]:
lr1 = LogisticRegression(solver='lbfgs', max_iter=1000)
lr1.fit(X_train_res, y_train_res.ravel())
predictions = lr1.predict(X_test)

In [58]:
submit = pd.DataFrame({'policy_id': X_test.index, 'TARGET': predictions})
submit.to_csv('../data/submit.csv')

In [61]:
submit['TARGET'].value_counts()

array([0., 1.])